In [ ]:
from datetime import timedelta, datetime


from airflow import DAG 
from airflow.utils.dates import days_ago
from airflow.operators.dummy_operator import DummyOperator
from airflow.providers.google.cloud.transfers.gcs_to_bigquery import GCSToBigQueryOperator
from airflow.providers.google.cloud.operators.bigquery import BigQueryCheckOperator
from airflow.contrib.operators.bigquery_operator import BigQueryOperator



GOOGLE_CONN_ID = "google_cloud_default"
PROJECT_ID="db-systems-group-project"
GS_PATH = "trips/"
BUCKET_NAME = 'master-data-bucket-group-5'
STAGING_DATASET = "trips_staging_dataset"
DATASET = "recipe_dataset"
LOCATION = "us (multiple regions in United States)"

default_args = {
    'owner': 'Koushik Modekurti',
    'depends_on_past': False,
    'email_on_failure': ['koushik.modekurti@sjsu.edu'],
    'email_on_retry': False,
    'retries': 1,
    'start_date':  days_ago(2),
    'retry_delay': timedelta(minutes=5),
}

with DAG('ETL_With_Airflow', schedule_interval=timedelta(days=1), default_args=default_args) as dag:
    start_pipeline = DummyOperator(
        task_id = 'start_pipeline',
        dag = dag
        )


    load_staging_dataset = DummyOperator(
        task_id = 'load_staging_dataset',
        dag = dag
        )    
    
    load_dataset_master_trip = GCSToBigQueryOperator(
        task_id = 'load_dataset_master_trip',
        bucket = BUCKET_NAME,
        source_objects = ['trips/Master_Trip_Data.csv'],
        destination_project_dataset_table = f'{PROJECT_ID}:{STAGING_DATASET}.dataset_master_trip',
        write_disposition='WRITE_TRUNCATE',
        source_format = 'csv',
        allow_quoted_newlines = 'true',
        skip_leading_rows = 1,
        schema_fields=[
        {'name': 'ride_id', 'type': 'STRING', 'mode': 'REQUIRED'},
        {'name': 'rideable_type', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'started_at', 'type': 'DATETIME', 'mode': 'NULLABLE'},
        {'name': 'ended_at', 'type': 'DATETIME', 'mode': 'NULLABLE'},
        {'name': 'start_station_id', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'end_station_id', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'start_lat', 'type': 'FLOAT', 'mode': 'NULLABLE'},
        {'name': 'start_lng', 'type': 'FLOAT', 'mode': 'NULLABLE'},
        {'name': 'end_lat', 'type': 'FLOAT', 'mode': 'NULLABLE'},
        {'name': 'end_lng', 'type': 'FLOAT', 'mode': 'NULLABLE'},
        {'name': 'member_casual', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'ride_length', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'start_date', 'type': 'DATE', 'mode': 'NULLABLE'},
        {'name': 'x', 'type': 'FLOAT', 'mode': 'NULLABLE'},
        {'name': 'y', 'type': 'FLOAT', 'mode': 'NULLABLE'},
        {'name': 'distance', 'type': 'FLOAT', 'mode': 'NULLABLE'},
        {'name': 'year', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'month', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'ride_len', 'type': 'STRING', 'mode': 'NULLABLE'},
            ]
        )

    load_dataset_ride_type = GCSToBigQueryOperator(
        task_id = 'load_dataset_ride_type',
        bucket = BUCKET_NAME,
        source_objects = ['trips/Ride_Type_Data.csv'],
        destination_project_dataset_table = f'{PROJECT_ID}:{STAGING_DATASET}.dataset_ride_type',
        write_disposition='WRITE_TRUNCATE',
        source_format = 'csv',
        allow_quoted_newlines = 'true',
        skip_leading_rows = 1,
        schema_fields=[
        {'name': 'Ride_Type', 'type': 'STRING', 'mode': 'REQUIRED'},
           {'name': 'Id', 'type': 'INTEGER', 'mode': 'REQUIRED'},
            ]
        )

    load_dataset_station = GCSToBigQueryOperator(
        task_id = 'load_dataset_station',
        bucket = BUCKET_NAME,
        source_objects = ['trips/Stations_Data.csv'],
        destination_project_dataset_table = f'{PROJECT_ID}:{STAGING_DATASET}.dataset_station',
        write_disposition='WRITE_TRUNCATE',
        source_format = 'csv',
        allow_quoted_newlines = 'true',
        skip_leading_rows = 1,
        schema_fields=[
        {'name': 'station', 'type': 'STRING', 'mode': 'REQUIRED'},
        {'name': 'station_id', 'type': 'STRING', 'mode': 'NULLABLE'},
            ]
        )
    
    load_dataset_rider_type = GCSToBigQueryOperator(
        task_id = 'load_dataset_rider_type',
        bucket = BUCKET_NAME,
        source_objects = ['trips/Rider_Type_Data.csv'],
        destination_project_dataset_table = f'{PROJECT_ID}:{STAGING_DATASET}.dataset_rider_type',
        write_disposition='WRITE_TRUNCATE',
        source_format = 'csv',
        allow_quoted_newlines = 'true',
        skip_leading_rows = 1,
        schema_fields=[
        {'name': 'Rider_Type', 'type': 'STRING', 'mode': 'REQUIRED'},
        {'name': 'Id', 'type': 'INTEGER', 'mode': 'NULLABLE'},
            ]
        )   
    
    check_dataset_master_trip = BigQueryCheckOperator(
        task_id = 'check_dataset_master_trip',
        use_legacy_sql=False,
        location = LOCATION,
        sql = f'SELECT count(*) FROM `{PROJECT_ID}.{STAGING_DATASET}.dataset_master_trip`'
        )

    check_dataset_ride_type = BigQueryCheckOperator(
        task_id = 'check_dataset_ride_type',
        use_legacy_sql=False,
        location = LOCATION,
        sql = f'SELECT count(*) FROM `{PROJECT_ID}.{STAGING_DATASET}.dataset_ride_type`'
        )

    check_dataset_station = BigQueryCheckOperator(
        task_id = 'check_dataset_station',
        use_legacy_sql=False,
        location = LOCATION,
        sql = f'SELECT count(*) FROM `{PROJECT_ID}.{STAGING_DATASET}.dataset_station`'
        ) 

    check_dataset_rider_type = BigQueryCheckOperator(
        task_id = 'check_dataset_rider_type',
        use_legacy_sql=False,
        location = LOCATION,
        sql = f'SELECT count(*) FROM `{PROJECT_ID}.{STAGING_DATASET}.dataset_rider_type`'
        )
 
    create_D_Table = DummyOperator(
        task_id = 'Create_D_Table',
        dag = dag
        )

    create_D_dataset_ayam = BigQueryOperator(
        task_id = 'create_D_dataset_ayam',
        use_legacy_sql = False,
        location = LOCATION,
        sql = './sql/D_dataset_ayam.sql'
        )

    create_D_dataset_ikan = BigQueryOperator(
        task_id = 'create_D_dataset_ikan',
        use_legacy_sql = False,
        location = LOCATION,
        sql = './sql/D_dataset_ikan.sql'
        )   

    create_D_dataset_tahu = BigQueryOperator(
        task_id = 'create_D_dataset_tahu',
        use_legacy_sql = False,
        location = LOCATION,
        sql = './sql/D_dataset_tahu.sql'
        )

    create_D_dataset_telur = BigQueryOperator(
        task_id = 'create_D_dataset_telur',
        use_legacy_sql = False,
        location = LOCATION,
        sql = './sql/D_dataset_telur.sql'
        )

    create_D_dataset_tempe = BigQueryOperator(
        task_id = 'create_D_dataset_tempe',
        use_legacy_sql = False,
        location = LOCATION,
        sql = './sql/D_dataset_tempe.sql'
        )         

    create_D_dataset_udang = BigQueryOperator(
        task_id = 'create_D_dataset_udang',
        use_legacy_sql = False,
        location = LOCATION,
        sql = './sql/D_dataset_udang.sql'
        )

    create_F_dataset_recipe = BigQueryOperator(
        task_id = 'create_F_dataset_recipe',
        use_legacy_sql = False,
        location = LOCATION,
        sql = './sql/F_dataset_recipe.sql'
        )

    check_F_dataset_recipe = BigQueryCheckOperator(
        task_id = 'check_F_dataset_recipe',
        use_legacy_sql=False,
        location = LOCATION,
        sql = f'SELECT count(*) FROM `{PROJECT_ID}.{DATASET}.F_dataset_recipe`'
        ) 

    finish_pipeline = DummyOperator(
        task_id = 'finish_pipeline',
        dag = dag
        ) 
start_pipeline >> load_staging_dataset

load_staging_dataset >> [load_dataset_ayam, load_dataset_ikan, load_dataset_tahu, load_dataset_telur, load_dataset_tempe, load_dataset_udang]

load_dataset_ayam >> check_dataset_ayam
load_dataset_ikan >> check_dataset_ikan
load_dataset_tahu >> check_dataset_tahu
load_dataset_telur >> check_dataset_telur
load_dataset_tempe >> check_dataset_tempe
load_dataset_udang >> check_dataset_udang

[check_dataset_ayam, check_dataset_ikan, check_dataset_tahu, check_dataset_telur, check_dataset_tempe, check_dataset_udang] >> create_D_Table

create_D_Table >> [create_D_dataset_ayam, create_D_dataset_ikan, create_D_dataset_tahu, create_D_dataset_telur, create_D_dataset_tempe, create_D_dataset_udang]

[create_D_dataset_ayam, create_D_dataset_ikan, create_D_dataset_tahu, create_D_dataset_telur, create_D_dataset_tempe, create_D_dataset_udang] >> create_F_dataset_recipe

create_F_dataset_recipe >> check_F_dataset_recipe >> finish_pipeline